In [1]:
#os..........

import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

* **sigma2:** Esta es la estimación de la varianza residual del modelo, que representa la variabilidad no explicada por los predictores en el modelo.
* **Ljung-Box (L1) (Q):** prueba de autocorrelación en el primer rezago (lag 1) de los residuales del modelo. Es una prueba de la independencia de los residuales. 
    * *Prob(Q):* es la probabilidad de obtener un valor de estadístico tan extremo como el observado si no hay autocorrelación en los residuales. El valor p es mayor que 0.05, lo que sugiere que no hay evidencia significativa de autocorrelación en el primer rezago de los residuales. Lo anterior indica que los residuales pueden ser considerados como ruido blanco o ruido aleatorio, lo que implica que el modelo es adecuado en términos de la autocorrelación de los residuales.


* **Jarque-Bera (JB):** Esto se refiere a la prueba de Jarque-Bera, que evalúa si los residuales del modelo siguen una distribución normal. Un valor p bajo (por debajo de un umbral común como 0.05) indica una desviación significativa de la normalidad.


* **Heteroskedasticity (H):** Esta estadística evalúa si la varianza de los residuales es constante a lo largo del tiempo. Un valor cercano a 1 sugiere homocedasticidad (varianza constante), mientras que valores significativamente diferentes de 1 pueden indicar heterocedasticidad (varianza variable).

* **Skew:** Muestra el sesgo de los residuales. Un valor positivo indica asimetría hacia la derecha, mientras que un valor negativo indica asimetría hacia la izquierda.


* **Kurtosis:** Mide la forma de la distribución de los residuales. Un valor alto de kurtosis indica colas pesadas en la distribución.

* **Prob(H) (two-sided):** prueba de heteroscedasticidad, que evalúa si la varianza de los residuales en el modelo es constante a lo largo del tiempo o si varía de manera significativa. En este contexto, "two-sided" significa que la prueba está evaluando la heteroscedasticidad de dos maneras posibles: si la varianza es significativamente mayor o significativamente menor que una constante.
        * Si el valor p es menor que este umbral, se considera que hay evidencia significativa de heteroscedasticidad en los residuales. Si el valor p es mayor que el umbral,  los residuales tienen una varianza constante.

In [2]:
#Libraries.............

import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go
from itertools import product
from tqdm import tqdm_notebook
import plotly.express as px


#sklearn
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

#skforecast
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.metrics import mean_absolute_error

# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX

# pmdarima
from pmdarima.arima import auto_arima

#darts
# from darts import TimeSeries
# from darts.utils.utils import SeasonalityMode, ModelMode
# from darts.models import ExponentialSmoothing


#prophet
# from prophet import Prophet
# from prophet.diagnostics import cross_validation, performance_metrics

# Modules from repo
from src.Utilities import utilities as tool

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.3f}'.format


#pip install git+https://github.com/JoaquinAmatRodrigo/skforecast#master

c:\Users\JENMOSQU\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using TensorFlow backend


In [4]:
df=pd.read_parquet('df.parquet')
df[['Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5']].head(6)

,Demanda_SIN_1,Demanda_SIN_2,Demanda_SIN_3,Demanda_SIN_4,Demanda_SIN_5
0,NaN,NaN,NaN,NaN,NaN
1,193865.750,NaN,NaN,NaN,NaN
2,184393.750,193865.750,NaN,NaN,NaN
3,167348.000,184393.750,193865.750,NaN,NaN
4,192717.880,167348.000,184393.750,193865.750,NaN
5,196696.350,192717.880,167348.000,184393.750,193865.750


In [5]:
#Import Data..............

df=pd.read_parquet('df.parquet')
columnas_indeseadas =['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D'] #columna con correlación 0 con la demanda
columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(1, 29)]
# columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(7, 29, 7)]
print("columnas rezagadas que no son necesarias porque skforecast rezaga internamente",columnas_indeseadas)
df = df.loc[:, ~df.columns.isin(columnas_indeseadas)]
print(df.head(1))
print("--------------------------------------------------------------------------------------------------")
print(df.tail(1))

columnas rezagadas que no son necesarias porque skforecast rezaga internamente ['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28']
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  precipitacion  Temperatura  HRA2_MN_D  HRA2_MX_D  THSM_MEDIA_D  THSM_MX_D  VV_10_MEDIA_D  VV_10_VECT_MEDIA_D  THSM_MN_D  PT_2_TT_D  PT_2_MX_D  PT_10_MX_D  PT_10_TT_D  TSSM_MEDIA_D  TSSM_MN_D  TSSM_MX_D  PTPG_TT_D  BSHG_TT_D  TPR_CAL
0 2017-12-01  VIERNES   193865.750      0             0       1          9.589       22.203  

In [6]:
#Correlación variables y filtrado........
#['THSM_MEDIA_D','PTPG_TT_D']
correlacion_con_demanda = df.corr()['Demanda_SIN']
nombres_variables = correlacion_con_demanda[(correlacion_con_demanda >= 0.2) | (correlacion_con_demanda <= -0.2)]
print(nombres_variables)
nombres_variables=nombres_variables.index.tolist()
nombres_variables +=['THSM_MEDIA_D'] #'PTPG_TT_D' #'VVAG_MEDIA_D'
nombres_variables.remove('Demanda_SIN')
df=df[['FECHA', 'TIPO_DIA', 'Demanda_SIN'] + nombres_variables]
print("columnas finales", df.columns)

Demanda_SIN           1.000
Covid                -0.246
Cambio_nivel          0.352
Tiempo                0.595
VV_10_MEDIA_D         0.213
VV_10_VECT_MEDIA_D    0.210
PT_10_TT_D           -0.291
Name: Demanda_SIN, dtype: float64
columnas finales Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D'], dtype='object')


In [7]:
#Rolling mean.........
columns_rolling = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D','PT_10_TT_D' ,'THSM_MEDIA_D']
window_size=60
df = tool.exploratory_tools.fill_null_values_with_rolling_mean(df, columns_rolling, window_size)
# df = df[df['FECHA'] <= pd.to_datetime('2023-09-30')

In [8]:
# Transformer one hot encoding and formatting-.......

print("--------------------------------------------------------------------------------------------------------------------------------------")
# ==============================================================================
one_hot_encoder = make_column_transformer(
                      #(StandardScaler(), ['Demanda_SIN']),
                      (   
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          ['TIPO_DIA'],
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

df = one_hot_encoder.fit_transform(df)

#-------------------------------------------------------------------------------------------------------------------------------------------
#Variable TIPO_DIA
#-------------------------------------------------------------------------------------------------------------------------------------------
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['FECHA'] + nombres_variables + columnas_tipo_dia
variable_dependiente = ['Demanda_SIN']
df = df[variable_dependiente + variables_exogenas] 

#-------------------------------------------------------------------------------------------------------------------------------------------
# Codificación de Fecha...
#-------------------------------------------------------------------------------------------------------------------------------------------
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df = df.set_index('FECHA')
df = df.asfreq('D')


df.head(2)

--------------------------------------------------------------------------------------------------------------------------------------


In [9]:
df['Demanda_SIN'].describe()

count     2130.000
mean    200514.685
std      15949.673
min     139832.380
25%     190976.908
50%     201522.504
75%     212219.054
max     240180.547
Name: Demanda_SIN, dtype: float64

In [16]:
# #Distribución de demanda por mes...

df['mes'] = df.index.month
fig = px.box(df, x='mes', y='Demanda_SIN', points="all")
fig.update_traces(marker=dict(size=2, color='#276981'), line=dict(width=1))
fig.update_layout(
    xaxis_title='Month',
    yaxis_title='Electricity Demand SIN kWh',
    title='Demand distribution by month',
)
fig.show()

In [20]:
#Demand distribution by day of the week............

df['dia_semana'] = df.index.dayofweek + 1

# Renombrar los nombres de los días de la semana para mostrarlos en el gráfico
day_names = {
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday',
    7: 'Sunday'
}
df['dia_semana'] = df['dia_semana'].map(day_names)

# Crear el boxplot utilizando Plotly
fig = px.box(df, x='dia_semana', y='Demanda_SIN', category_orders={"dia_semana": ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']}, points="all")
fig.update_traces(marker=dict(size=2, color='#276981'), line=dict(width=1))
fig.update_layout(
    xaxis_title='Day of the week',
    yaxis_title='Electricity Demand SIN kWh',
    title='Demand distribution by day of the week',
)
fig.show()

In [24]:
df.resample('Y')['Demanda_SIN'].describe()

,count,mean,std,min,25%,50%,75%,max
FECHA,,,,,,,,
2017-12-31,31.000,182862.066,13055.974,149585.800,176626.505,184393.750,193353.095,197097.780
2018-12-31,365.000,189234.523,12116.407,139832.380,184481.060,193096.200,197961.870,205851.800
2019-12-31,365.000,196886.133,11912.643,147133.360,191718.100,200277.300,204416.710,216914.160
2020-12-31,366.000,192235.953,13878.990,155493.123,181562.276,194864.911,203413.597,217049.027
2021-12-31,365.000,202876.309,12283.617,155622.822,197172.511,206129.062,211734.875,220384.534
2022-12-31,365.000,209862.063,11634.794,165336.259,206391.243,213984.865,217909.664,228615.713
2023-12-31,273.000,217896.158,14153.749,165197.797,211313.495,220936.675,227657.679,240180.547


* **adf_statistic:** El valor estadístico de la prueba ADF. Si este valor es menor que los valores críticos correspondientes, se puede rechazar la hipótesis nula de que la serie de tiempo tiene una raíz unitaria y se puede concluir que la serie de tiempo es estacionaria. Si el valor estadístico es mayor que los valores críticos, se debe aceptar la hipótesis nula y se concluye que la serie de tiempo no es estacionaria.

* **p_value:** El valor p de la prueba ADF. Si este valor es menor que el nivel de significancia elegido (por defecto 0.05), se puede rechazar la hipótesis nula y se concluye que la serie de tiempo es estacionaria. Si el valor p es mayor que el nivel de significancia, se debe aceptar la hipótesis nula y se concluye que la serie de tiempo no es estacionaria.

* **used_lags:** El número de rezagos utilizado en la prueba ADF.
* **n_observations:** El número de observaciones en la serie de tiempo.
* **critical_values:** Los valores críticos para diferentes niveles de significancia. Si el valor estadístico de la prueba ADF es menor que el valor crítico correspondiente, se puede concluir que la serie de tiempo es estacionaria.

In [8]:
# Evalúa si una serie de tiempo tiene una raíz unitaria

tool.demand_utilities.test_stationarity(df)

{'Demanda_SIN': {'adf_statistic': -2.30382229267568,
  'p_value': 0.17073816943395326,
  'used_lags': 26,
  'n_observations': 2103,
  'critical_values': {'1%': -3.4334633142611697,
   '5%': -2.862915331606178,
   '10%': -2.567502161536324}},
 'Covid': {'adf_statistic': -1.677787210145107,
  'p_value': 0.44259197546756174,
  'used_lags': 0,
  'n_observations': 2129,
  'critical_values': {'1%': -3.4334252475234646,
   '5%': -2.8628985239834686,
   '10%': -2.567493212493555}},
 'Cambio_nivel': {'adf_statistic': -1.2401344158116254,
  'p_value': 0.6560971294054155,
  'used_lags': 0,
  'n_observations': 2129,
  'critical_values': {'1%': -3.4334252475234646,
   '5%': -2.8628985239834686,
   '10%': -2.567493212493555}},
 'Tiempo': {'adf_statistic': 42.401988336668055,
  'p_value': 1.0,
  'used_lags': 21,
  'n_observations': 2108,
  'critical_values': {'1%': -3.4334559207180577,
   '5%': -2.8629120671487107,
   '10%': -2.5675004234073504}},
 'VV_10_MEDIA_D': {'adf_statistic': -1.56583399553426

In [9]:
from scipy.stats import friedmanchisquare

def FriedmanSeasonality_test(dataframe, column):
    ts = dataframe[column]
    seasons = np.unique(ts.index.strftime('%B'))
    min_length = min(len(ts[ts.index.month == i+1]) for i in range(len(seasons)))
    seasons_data = {season: ts[ts.index.month == i+1][:min_length] for i, season in enumerate(seasons)}
    f_value, p_value = friedmanchisquare(*seasons_data.values())
    if p_value <= 0.05:
        print(f"El p-valor es {p_value}, por lo tanto, se puede inferir que hay patrones estacionales significativos.")
    else:
        print(f"El p-valor es {p_value}, por lo tanto, no se puede inferir que hay patrones estacionales significativos.")
    return f_value, p_value

# # Generador
FriedmanSeasonality_test(df, 'Demanda_SIN')

El p-valor es 6.795737146435293e-38, por lo tanto, se puede inferir que hay patrones estacionales significativos.


(205.00615384615412, 6.795737146435293e-38)

In [10]:
# Separación datos train-val-test................
#-------------------------------------------------------------------------------------------------------------------------------------------

datos = df[df.index >= '2017-12-01']
fin_train = '2023-05-31'
fin_validacion = '2023-09-23'

datos_train = datos[datos.index <= fin_train].copy()
datos_val = datos[(datos.index > fin_train) & (datos.index <= fin_validacion)].copy()
datos_test = datos[datos.index > fin_validacion].copy()

print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas validacion : {datos_val.index.min()} --- {datos_val.index.max()}  (n={len(datos_val)})")
print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
print("-------------------------------------------------------------------------------------------------------------------------------------")


Fechas train      : 2017-12-01 00:00:00 --- 2023-05-31 00:00:00  (n=2008)
Fechas validacion : 2023-06-01 00:00:00 --- 2023-09-23 00:00:00  (n=115)
Fechas test       : 2023-09-24 00:00:00 --- 2023-09-30 00:00:00  (n=7)
-------------------------------------------------------------------------------------------------------------------------------------


In [54]:
import plotly.express as px

# Crear un gráfico de serie temporal con Plotly
fig = px.line(datos, x=datos.index, y='Demanda_SIN',
              title='Electricity Demand Split',
              labels={'Demanda_SIN': 'Electricity Demand SIN kWh'},
              color_discrete_map={datos.index.year[0]: '#bc8f8f'})

# Añadir etiquetas de las particiones de datos (entrenamiento, validación, test)
fig.add_shape(type="rect", x0=datos_train.index[0], x1=datos_train.index[-1],
              y0=0, y1=1, line=dict(color='#bc8f8f', width=2),
              xref='x', yref='paper', fillcolor='#bc8f8f', opacity=0.3,
              name='Training')

fig.add_shape(type="rect", x0=datos_val.index[0], x1=datos_val.index[-1],
              y0=0, y1=1, line=dict(color='#276981', width=2),
              xref='x', yref='paper', fillcolor='#276981', opacity=0.3,
              name='Validation')

fig.add_shape(type="rect", x0=datos_test.index[0], x1=datos_test.index[-1],
              y0=0, y1=1, line=dict(color="green", width=2),
              xref='x', yref='paper', fillcolor="green", opacity=0.3,
              name='Testing')

# Añadir líneas indicadoras de colores y labels fuera del gráfico
fig.update_layout(
    shapes=[
        dict(
            type="rect",
            x0=datos_train.index[0],
            x1=datos_train.index[-1],
            y0=0,
            y1=1,
            line=dict(color='#bc8f8f', width=2),
            xref='x',
            yref='paper',
            fillcolor='#bc8f8f',
            opacity=0.3,
            name='Training'
        ),
        dict(
            type="rect",
            x0=datos_val.index[0],
            x1=datos_val.index[-1],
            y0=0,
            y1=1,
            line=dict(color='#276981', width=2),
            xref='x',
            yref='paper',
            fillcolor='#276981',
            opacity=0.3,
            name='Validation'
        ),
        dict(
            type="rect",
            x0=datos_test.index[0],
            x1=datos_test.index[-1],
            y0=0,
            y1=1,
            line=dict(color="green", width=2),
            xref='x',
            yref='paper',
            fillcolor="green",
            opacity=0.3,
            name='Testing'
        ),
        dict(
            type="line",
            x0=datos_train.index[-1] + pd.DateOffset(days=2),
            x1=datos_train.index[-1] + pd.DateOffset(days=5),
            y0=1.1,
            y1=1.1,
            xref='x',
            yref='paper',
            line=dict(color='#bc8f8f', width=2, dash='solid')
        ),
        dict(
            type="line",
            x0=datos_val.index[-1] + pd.DateOffset(days=2),
            x1=datos_val.index[-1] + pd.DateOffset(days=5),
            y0=1.1,
            y1=1.1,
            xref='x',
            yref='paper',
            line=dict(color='#276981', width=2, dash='solid')
        ),
        dict(
            type="line",
            x0=datos_test.index[-1] + pd.DateOffset(days=2),
            x1=datos_test.index[-1] + pd.DateOffset(days=5),
            y0=1.1,
            y1=1.1,
            xref='x',
            yref='paper',
            line=dict(color="green", width=2, dash='solid')
        )
    ],
    annotations=[
        dict(
            x=datos_train.index[-1] + pd.DateOffset(days=5),
            y=1.1,
            xref="x",
            yref="paper",
            text="Training Data",
            showarrow=False,
            font=dict(size=10, color='#bc8f8f'),
            xanchor="center"
        ),
        dict(
            x=datos_val.index[-1] + pd.DateOffset(days=5),
            y=1.1,
            xref="x",
            yref="paper",
            text="Validation Data",
            showarrow=False,
            font=dict(size=10, color='#276981'),
            xanchor="center"
        ),
        dict(
            x=datos_test.index[-1] + pd.DateOffset(days=5),
            y=1.1,
            xref="x",
            yref="paper",
            text="Testing Data",
            showarrow=False,
            font=dict(size=10, color="green"),
            xanchor="center"
        )
    ]
)

# Ajustar el margen inferior para dejar espacio para
fig.update_layout(margin=dict(b=50))

# Cambiar el nombre del eje x a "Date"
fig.update_xaxes(title_text='Date-daily')

# Mostrar el gráfico
fig.show()


In [21]:
# Graficar Demanda....

df['Año'] = df['FECHA'].dt.year

# Crear un gráfico de serie temporal con Plotly
fig = px.line(df, x='FECHA', y='Demanda_SIN', color='Año',
              title='Demanda eléctrica',
              labels={'Demanda_SIN': 'Demanda'})

# Mostrar el gráfico
fig.show()

In [11]:
#CREACIÓN DE TENDENCIA POR AÑO.... IDEA
# # Lista de columnas a excluir que no son significativas en el modelo escogido...
# #columnas_a_excluir = ['TIPO_DIA_2--ENE', 'TIPO_DIA_DOALFENE', 'TIPO_DIA_DOVDIC','TIPO_DIA_DOVENE']
# #df = df.drop(columns=columnas_a_excluir)
# df['FECHA'] = pd.to_datetime(df['FECHA'])
# # Crear la columna "Tendencia"
# df['Tendencia'] = df['FECHA'].dt.year.groupby(df['FECHA'].dt.year).cumcount() + 1


In [78]:
#ACF.......................................

from statsmodels.tsa.stattools import acf

# Calcular la función de autocorrelación
lags = len(df['Demanda_SIN']) - 1
acf_values = acf(df['Demanda_SIN'], nlags=lags)

# Crear un gráfico de barras con Plotly
fig = go.Figure()

fig.add_trace(go.Bar(x=np.arange(0, lags+1), y=acf_values,
                     marker_color='#276981',
                     opacity=0.7,
                     name='Autocorrelation'))

# Configuraciones del diseño del gráfico
fig.update_layout(
    title=" Autocorrelation Function (ACF)",
    xaxis=dict(title='Lags'),
    yaxis=dict(title="Autocorrelation"),
    showlegend=True
)

# Mostrar el gráfico
fig.show()

In [10]:
#PACF...........................

from statsmodels.tsa.stattools import pacf

# Calcular la función de autocorrelación parcial
lags = len(df['Demanda_SIN']) - 1
pacf_values = pacf(df['Demanda_SIN'], nlags=500)

# Crear un gráfico de barras con Plotly
fig = go.Figure()

fig.add_trace(go.Bar(x=np.arange(1, lags+1), y=pacf_values[1:],
                     marker_color='#276981',
                     opacity=0.7,
                     name='Partial Autocorrelation'))

# Configuraciones del diseño del gráfico
fig.update_layout(
    title='Partial Autocorrelation Function (PACF)',
    xaxis=dict(title='Lags'),
    yaxis=dict(title='Partial Autocorrelation'),
    showlegend=True
)

# Mostrar el gráfico
fig.show()

In [74]:
datos_train.head(2)

,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-12-01,193865.750,0,0,1,1.703,0.935,6.232,19.113,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
2017-12-02,184393.750,0,0,2,1.757,0.930,7.888,19.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


### Auto Arima

In [75]:
nombres_variables

['Covid',
 'Cambio_nivel',
 'Tiempo',
 'VV_10_MEDIA_D',
 'VV_10_VECT_MEDIA_D',
 'PT_10_TT_D',
 'THSM_MEDIA_D']

In [76]:
columnas_tipo_dia

['TIPO_DIA_1--ENE',
 'TIPO_DIA_1--MAY',
 'TIPO_DIA_2--ENE',
 'TIPO_DIA_20--JUL',
 'TIPO_DIA_24--DIC',
 'TIPO_DIA_25--DIC',
 'TIPO_DIA_31--DIC',
 'TIPO_DIA_7--AGO',
 'TIPO_DIA_8--DIC',
 'TIPO_DIA_DOALF',
 'TIPO_DIA_DOALFENE',
 'TIPO_DIA_DOMINGO',
 'TIPO_DIA_DOVDIC',
 'TIPO_DIA_DOVENE',
 'TIPO_DIA_DSS',
 'TIPO_DIA_JSS',
 'TIPO_DIA_JUEVES',
 'TIPO_DIA_JUVDIC',
 'TIPO_DIA_JUVENE',
 'TIPO_DIA_LF',
 'TIPO_DIA_LFENE',
 'TIPO_DIA_LUNES',
 'TIPO_DIA_LUVDIC',
 'TIPO_DIA_LUVENE',
 'TIPO_DIA_MADLF',
 'TIPO_DIA_MARTES',
 'TIPO_DIA_MAVDIC',
 'TIPO_DIA_MAVENE',
 'TIPO_DIA_MIERCOLES',
 'TIPO_DIA_MIVDIC',
 'TIPO_DIA_MIVENE',
 'TIPO_DIA_MSS',
 'TIPO_DIA_SAALF',
 'TIPO_DIA_SAALFENE',
 'TIPO_DIA_SABADO',
 'TIPO_DIA_SAVDIC',
 'TIPO_DIA_SAVENE',
 'TIPO_DIA_SSS',
 'TIPO_DIA_VIERNES',
 'TIPO_DIA_VIVDIC',
 'TIPO_DIA_VIVENE',
 'TIPO_DIA_VSS']

In [77]:
#Con todas las variables........

variables_exogenas =  nombres_variables + columnas_tipo_dia
arima=auto_arima(datos_train['Demanda_SIN'], X=datos_train[variables_exogenas], start_p=1, d=1, D=1,  max_p=3, max_d=2 ,max_q=3, m=7, start_P=0, 
           seasonal=True, seasonal_test='ocsb',trace=True,scoring='mse', error_action='ignore', suppress_warnings=True, stepwise=True)

print("BIC",arima.bic())
print("AIC",arima.aic())
#arima.plot_diagnostics()
arima.summary()

Performing stepwise search to minimize aic
 ARIMA(1,1,2)(0,1,1)[7]             : AIC=37440.976, Time=9.89 sec
 ARIMA(0,1,0)(0,1,0)[7]             : AIC=51164.354, Time=0.91 sec
 ARIMA(1,1,0)(1,1,0)[7]             : AIC=37663.413, Time=16.67 sec
 ARIMA(0,1,1)(0,1,1)[7]             : AIC=37475.756, Time=7.41 sec
 ARIMA(1,1,2)(0,1,0)[7]             : AIC=inf, Time=30.44 sec
 ARIMA(1,1,2)(1,1,1)[7]             : AIC=inf, Time=42.01 sec
 ARIMA(1,1,2)(0,1,2)[7]             : AIC=inf, Time=77.13 sec
 ARIMA(1,1,2)(1,1,0)[7]             : AIC=inf, Time=37.28 sec
 ARIMA(1,1,2)(1,1,2)[7]             : AIC=inf, Time=73.11 sec
 ARIMA(0,1,2)(0,1,1)[7]             : AIC=37450.125, Time=7.21 sec
 ARIMA(1,1,1)(0,1,1)[7]             : AIC=37371.253, Time=38.38 sec
 ARIMA(1,1,1)(0,1,0)[7]             : AIC=inf, Time=15.90 sec
 ARIMA(1,1,1)(1,1,1)[7]             : AIC=inf, Time=48.64 sec
 ARIMA(1,1,1)(0,1,2)[7]             : AIC=inf, Time=59.18 sec
 ARIMA(1,1,1)(1,1,0)[7]             : AIC=inf, Time=35.06

<class 'statsmodels.iolib.summary.Summary'>
"""
                                       SARIMAX Results                                        
==============================================================================================
Dep. Variable:                                      y   No. Observations:                 2008
Model:             SARIMAX(3, 1, 0)x(0, 1, [1, 2], 7)   Log Likelihood              -18551.230
Date:                                Sun, 12 Nov 2023   AIC                          37212.461
Time:                                        13:50:34   BIC                          37520.511
Sample:                                    12-01-2017   HQIC                         37325.570
                                         - 05-31-2023                                         
Covariance Type:                                  opg                                         
======================================================================================
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Covid               6452.8968   8962.022      0.720      0.472   -1.11e+04     2.4e+04
Cambio_nivel       -6357.7474   1.06e+04     -0.598      0.550   -2.72e+04    1.45e+04
Tiempo                -7.9872   2082.596     -0.004      0.997   -4089.801    4073.827
VV_10_MEDIA_D         17.5348     87.142      0.201      0.841    -153.260     188.330
VV_10_VECT_MEDIA_D   -21.8639     87.957     -0.249      0.804    -194.256     150.528
PT_10_TT_D          -239.9330     24.981     -9.604      0.000    -288.896    -190.970
THSM_MEDIA_D        3402.1683    265.718     12.804      0.000    2881.370    3922.967
TIPO_DIA_1--ENE    -2.377e+04   7306.619     -3.253      0.001   -3.81e+04   -9446.617
TIPO_DIA_1--MAY    -9069.5782   7256.823     -1.250      0.211   -2.33e+04    5153.534
TIPO_DIA_2--ENE      -30.7399   7330.979     -0.004      0.997   -1.44e+04    1.43e+04
TIPO_DIA_20--JUL   -5037.8649   7282.834     -0.692      0.489   -1.93e+04    9236.228
TIPO_DIA_24--DIC   -3130.2903   7211.191     -0.434      0.664   -1.73e+04     1.1e+04
TIPO_DIA_25--DIC   -2.055e+04   7285.435     -2.821      0.005   -3.48e+04   -6274.926
TIPO_DIA_31--DIC   -4426.8821   7224.088     -0.613      0.540   -1.86e+04    9732.071
TIPO_DIA_7--AGO    -6048.3792   7258.578     -0.833      0.405   -2.03e+04    8178.172
TIPO_DIA_8--DIC    -8026.9561   7125.012     -1.127      0.260    -2.2e+04    5937.810
TIPO_DIA_DOALF     -6108.2267   7250.136     -0.842      0.400   -2.03e+04    8101.779
TIPO_DIA_DOALFENE  -2244.7210   7598.067     -0.295      0.768   -1.71e+04    1.26e+04
TIPO_DIA_DOMINGO   -4246.9411   7215.455     -0.589      0.556   -1.84e+04    9895.091
TIPO_DIA_DOVDIC     -897.4410   7338.637     -0.122      0.903   -1.53e+04    1.35e+04
TIPO_DIA_DOVENE    -2565.3701   7400.136     -0.347      0.729   -1.71e+04    1.19e+04
TIPO_DIA_DSS       -8729.4440   7314.771     -1.193      0.233   -2.31e+04    5607.244
TIPO_DIA_JSS       -1.785e+04   7181.901     -2.486      0.013   -3.19e+04   -3775.404
TIPO_DIA_JUEVES     9055.2788   7087.103      1.278      0.201   -4835.188    2.29e+04
TIPO_DIA_JUVDIC     9397.4447   7014.991      1.340      0.180   -4351.685    2.31e+04
TIPO_DIA_JUVENE     9008.9581   7163.923      1.258      0.209   -5032.073     2.3e+04
TIPO_DIA_LF        -8459.7895   7214.188     -1.173      0.241   -2.26e+04    5679.759
TIPO_DIA_LFENE     -7336.8935   7389.490     -0.993      0.321   -2.18e+04    7146.241
TIPO_DIA_LUNES      1.383e+04   7183.712      1.925      0.054    -249.366    2.79e+04
TIPO_DIA_LUVDIC     1.192e+04   7276.878      1.639      0.101   -2338.210    2.62e+04
TIPO_DIA_LUVENE     9075.8571   7373.296      1.231      0.218   -5375.537    2.35e+04
TIPO_DIA_MADLF      8801.8437   7235.530      1.216      0.224   -5379.534     2.3e+04
TIPO_DIA_MARTES     1.121e+04   7227.869      1.551 

In [9]:
#demanda SIN_Imputada y con todas solo el covid y variable, tiempo y las variables de clima (me quedo con esta estimación)
#ARIMAX(1, 1, 3)x(2, 1, [], 7)
variables_exogenas = ['Covid', 'Tiempo', 'PTPG_TT_D', 'TSSM_MEDIA_D', 'THSM_MEDIA_D', 'VVAG_MEDIA_D'] + columnas_tipo_dia
y = df['Demanda_SIN']
x=df[variables_exogenas]
d=1
D=1
s=7
parametros = [3,3,3,3]
validacion = 28
forecasting_steps=28
result_1=tool.modeling.optimize_SARIMA(y, x,  d, D, s, parametros, validacion, forecasting_steps)
result_1

  0%|          | 0/256 [00:00<?, ?it/s]

{'forecast': 2023-08-01   232426.43
 2023-08-02   235652.77
 2023-08-03   234357.84
 2023-08-04   233075.04
 2023-08-05   221940.59
 2023-08-06   206410.76
 2023-08-07   230269.49
 2023-08-08   234815.56
 2023-08-09   235252.75
 2023-08-10   237488.34
 2023-08-11   234833.94
 2023-08-12   226262.20
 2023-08-13   204817.59
 2023-08-14   228787.79
 2023-08-15   236387.54
 2023-08-16   235522.64
 2023-08-17   222241.00
 2023-08-18   236681.16
 2023-08-19   224702.63
 2023-08-20   204048.63
 2023-08-21   230438.05
 2023-08-22   233695.38
 2023-08-23   234900.13
 2023-08-24   234925.53
 2023-08-25   237362.40
 2023-08-26   226861.58
 2023-08-27   204110.18
 2023-08-28   229489.81
 Freq: D, Name: predicted_mean, dtype: float64,
 'parameters': (1, 1, 0, 1),
 'mape': 1.6125648545432907,
 'rmse': 4590.07951418726}

### Grid search hyperparameter

In [30]:

# Grid search hyperparameter
# ======================================================================================
forecaster = ForecasterSarimax(
                 regressor=Sarimax(order=(1, 1, 2), seasonal_order=(0, 1, 2, 7), maxiter=200),
             )


param_grid = {'order': [(1, 0, 1), (1, 1, 2), (2, 1, 1), (1, 1, 1), (2, 1, 2)],
             'seasonal_order': [(0, 1, 2, 7), (1, 1, 2, 7)],
             'trend': [None, 'n', 'c']}

results_grid = grid_search_sarimax(
                forecaster                  = forecaster,
                y                           = datos.loc[:fin_validacion, 'Demanda_SIN'],
                exog                        = datos.loc[:fin_validacion, variables_exogenas],
                param_grid                  = param_grid,
                initial_train_size          = len(datos.loc[:fin_train]),
                fixed_train_size            = False,
                steps                       = len(datos.loc[fin_train:fin_validacion]),
                metric                      = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                refit                       = False,
                verbose                     = False
               )

results_grid

Number of models compared: 15.


params grid: 100%|██████████| 15/15 [10:03<00:00, 40.23s/it]


`Forecaster` refitted using the best-found parameters, and the whole data set: 
  Parameters: {'order': (12, 0, 0), 'seasonal_order': (0, 0, 0, 7), 'trend': None}
  Backtesting metric: 19226855.682752583



,params,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error,mean_squared_log_error,order,seasonal_order,trend
0,"{'order': (12, 0, 0), 'seasonal_order': (0, 0, 0, 7), 'trend': None}",19226855.68,3514.96,0.02,0.00,"(12, 0, 0)","(0, 0, 0, 7)",None
1,"{'order': (12, 0, 0), 'seasonal_order': (0, 0, 0, 7), 'trend': 'n'}",19226855.68,3514.96,0.02,0.00,"(12, 0, 0)","(0, 0, 0, 7)",n
12,"{'order': (14, 1, 4), 'seasonal_order': (0, 0, 0, 7), 'trend': None}",91861069.67,5750.08,0.03,0.00,"(14, 1, 4)","(0, 0, 0, 7)",None
13,"{'order': (14, 1, 4), 'seasonal_order': (0, 0, 0, 7), 'trend': 'n'}",91861069.67,5750.08,0.03,0.00,"(14, 1, 4)","(0, 0, 0, 7)",n
14,"{'order': (14, 1, 4), 'seasonal_order': (0, 0, 0, 7), 'trend': 'c'}",92464084.24,5766.58,0.03,0.00,"(14, 1, 4)","(0, 0, 0, 7)",c
9,"{'order': (12, 1, 1), 'seasonal_order': (0, 0, 0, 7), 'trend': None}",94334067.94,5777.80,0.03,0.00,"(12, 1, 1)","(0, 0, 0, 7)",None
10,"{'order': (12, 1, 1), 'seasonal_order': (0, 0, 0, 7), 'trend': 'n'}",94334067.94,5777.80,0.03,0.00,"(12, 1, 1)","(0, 0, 0, 7)",n
6,"{'order': (12, 1, 0), 'seasonal_order': (0, 0, 0, 7), 'trend': None}",94355579.16,5778.24,0.03,0.00,"(12, 1, 0)","(0, 0, 0, 7)",None
7,"{'order': (12, 1, 0), 'seasonal_order': (0, 0, 0, 7), 'trend': 'n'}",94355579.16,5778.24,0.03,0.00,"(12, 1, 0)","(0, 0, 0, 7)",n
11,"{'order': (12, 1, 1), 'seasonal_order': (0, 0, 0, 7), 'trend': 'c'}",94673965.78,5785.76,0.03,0.00,"(12, 1, 1)","(0, 0, 0, 7)",c


In [31]:
forecaster

ForecasterSarimax 
Regressor: Sarimax(12,0,0)(0,0,0)[7] 
Regressor parameters: {'concentrate_scale': False, 'dates': None, 'disp': False, 'enforce_invertibility': True, 'enforce_stationarity': True, 'freq': None, 'hamilton_representation': False, 'maxiter': 200, 'measurement_error': False, 'method': 'lbfgs', 'missing': 'none', 'mle_regression': True, 'order': (12, 0, 0), 'seasonal_order': (0, 0, 0, 7), 'simple_differencing': False, 'sm_fit_kwargs': {}, 'sm_init_kwargs': {}, 'sm_predict_kwargs': {}, 'start_params': None, 'time_varying_regression': False, 'trend': None, 'trend_offset': 1, 'use_exact_diffuse': False, 'validate_specification': True} 
fit_kwargs: {} 
Window size: 1 
Transformer for y: None 
Transformer for exog: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'VVAG_MEDIA_D', 'PT_10_TT_D', 'TIPO_DIA_1--ENE', 'TIPO_DIA_1--MAY

### Backtesting_sarimax
* ORDEN ESCOGIDO DESPUÉS DE ITERAR CON DIFERENTES COMBINACIONES DE COLUMNAS: SARIMAX(1, 1, 2)x(0, 1, 2, 7)

In [32]:
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas =  nombres_variables + columnas_tipo_dia

# Predicciones de backtesting con el mejor modelo según auto arima
# ==============================================================================
forecaster = ForecasterSarimax(
                 regressor=Sarimax(order=(3, 1, 3), seasonal_order=(0, 1, 2, 7), maxiter=200),
                 transformer_y = StandardScaler()
             )

#datos['Demanda_SIN'].index = pd.to_datetime(datos['Demanda_SIN'].index)

# datos['Demanda_SIN'].index.freq = 'D'

metric, predictions = backtesting_sarimax(
                          forecaster            = forecaster,
                          y                     = datos.loc[:fin_validacion, 'Demanda_SIN'],
                          exog                  = datos.loc[:fin_validacion, variables_exogenas],
                          initial_train_size    = len(datos.loc[:fin_train]),
                          fixed_train_size      = False,
                          allow_incomplete_fold = True,
                          steps                 = len(datos.loc[fin_train:fin_validacion]),
                          metric                = ['mean_squared_error', 'mean_absolute_error','mean_absolute_percentage_error'],
                          refit                 = True,
                          n_jobs                = 1,
                          interval              = [2.5, 97.5],
                          suppress_warnings_fit = True,
                          verbose               = True,
                          show_progress         = True
                      )

print(f"Error backtest: {metric}")
predictions.head(4)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 1947
Number of observations used for backtesting: 91
    Number of folds: 1
    Number of steps per fold: 92
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 91 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-03-31 00:00:00  (n=1947)
    Validation: 2023-04-01 00:00:00 -- 2023-06-30 00:00:00  (n=91)



100%|██████████| 1/1 [00:00<00:00, 30.46it/s]

Error backtest: [52491686.532280475, 4718.582683580012, 0.02169363294407355]


,pred,lower_bound,upper_bound
2023-04-01,215301.02,210641.98,219960.05
2023-04-02,194679.41,188364.94,200993.87
2023-04-03,219869.97,212798.23,226941.71
2023-04-04,222402.07,214861.97,229942.17


In [ ]:

# Feature importances
# ==============================================================================
forecaster.get_feature_importances()

### Alternativas

In [13]:
#Solo el clima y el TIPO_DIA.........

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['PTPG_TT_D', 'TSSM_MEDIA_D', 'THSM_MEDIA_D', 'VVAG_MEDIA_D'] + columnas_tipo_dia
fin_validacion = '2023-04-30'
# Predicciones de backtesting con el mejor modelo según auto arima
# ==============================================================================
forecaster = ForecasterSarimax(
                 regressor=Sarimax(order=(1, 1, 2), seasonal_order=(0, 1, 2, 7), maxiter=200),
             )

metric, predictions = backtesting_sarimax(
                          forecaster            = forecaster,
                          y                     = df['Demanda_SIN'],
                          exog                  = df[variables_exogenas],
                          initial_train_size    = len(df.loc[:fin_validacion]),
                          fixed_train_size      = True,
                          steps                 = 7,
                          metric                = ['mean_squared_error', 'mean_absolute_error','mean_absolute_percentage_error'],
                          refit                 = True,
                          n_jobs                = 'auto',
                          suppress_warnings_fit = True,
                          verbose               = True,
                          show_progress         = True
                      )

print(f"Error backtest: {metric}")
predictions.head(4)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 1977
Number of observations used for backtesting: 92
    Number of folds: 14
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 1 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-04-30 00:00:00  (n=1977)
    Validation: 2023-05-01 00:00:00 -- 2023-05-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-08 00:00:00 -- 2023-05-07 00:00:00  (n=1977)
    Validation: 2023-05-08 00:00:00 -- 2023-05-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-15 00:00:00 -- 2023-05-14 00:00:00  (n=1977)
    Validation: 2023-05-15 00:00:00 -- 2023-05-21 00:00:00  (n=7)
Fold: 3
    Training:   2017-12-22 00:00:00 -- 2023-05-21 00:00:00  (n=1977)
    Validation: 2023-05-22 00:00:00 -- 2023-05-28 00:00:00  (n=7)
Fold: 4
    Training:   2017-12-29 00:00:00 -- 2023-05-28 00:00:00  (n=

  0%|          | 0/14 [00:00<?, ?it/s]

Error backtest: [13762195.91704583, 2869.3963370632123, 0.013071479935981189]


,pred
2023-05-01,199132.51
2023-05-02,223144.34
2023-05-03,225004.73
2023-05-04,226242.05


In [14]:
#Solo el covid, tendencia y el TIPO_DIA.........

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['Covid', 'Tiempo'] + columnas_tipo_dia
fin_validacion = '2023-04-30'
# Predicciones de backtesting con el mejor modelo según auto arima
# ==============================================================================
forecaster = ForecasterSarimax(
                 regressor=Sarimax(order=(1, 1, 2), seasonal_order=(0, 1, 2, 7), maxiter=200),
             )

metric, predictions = backtesting_sarimax(
                          forecaster            = forecaster,
                          y                     = df['Demanda_SIN'],
                          exog                  = df[variables_exogenas],
                          initial_train_size    = len(df.loc[:fin_validacion]),
                          fixed_train_size      = True,
                          steps                 = 7,
                          metric                = ['mean_squared_error', 'mean_absolute_error','mean_absolute_percentage_error'],
                          refit                 = True,
                          n_jobs                = 'auto',
                          suppress_warnings_fit = True,
                          verbose               = True,
                          show_progress         = True
                      )

print(f"Error backtest: {metric}")
predictions.head(4)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 1977
Number of observations used for backtesting: 92
    Number of folds: 14
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 1 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-04-30 00:00:00  (n=1977)
    Validation: 2023-05-01 00:00:00 -- 2023-05-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-08 00:00:00 -- 2023-05-07 00:00:00  (n=1977)
    Validation: 2023-05-08 00:00:00 -- 2023-05-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-15 00:00:00 -- 2023-05-14 00:00:00  (n=1977)
    Validation: 2023-05-15 00:00:00 -- 2023-05-21 00:00:00  (n=7)
Fold: 3
    Training:   2017-12-22 00:00:00 -- 2023-05-21 00:00:00  (n=1977)
    Validation: 2023-05-22 00:00:00 -- 2023-05-28 00:00:00  (n=7)
Fold: 4
    Training:   2017-12-29 00:00:00 -- 2023-05-28 00:00:00  (n=

  0%|          | 0/14 [00:00<?, ?it/s]

Error backtest: [21572107.434138097, 3495.6696672270677, 0.015854404542672614]


,pred
2023-05-01,196216.47
2023-05-02,219680.48
2023-05-03,221046.80
2023-05-04,221956.05


In [15]:
# Solo el covid y el TIPO_DIA.........

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['Covid'] + columnas_tipo_dia
fin_validacion = '2023-04-30'
# Predicciones de backtesting con el mejor modelo según auto arima
# ==============================================================================
forecaster = ForecasterSarimax(
                 regressor=Sarimax(order=(1, 1, 2), seasonal_order=(0, 1, 2, 7), maxiter=200),
             )

metric, predictions = backtesting_sarimax(
                          forecaster            = forecaster,
                          y                     = df['Demanda_SIN'],
                          exog                  = df[variables_exogenas],
                          initial_train_size    = len(df.loc[:fin_validacion]),
                          fixed_train_size      = True,
                          steps                 = 7,
                          metric                = ['mean_squared_error', 'mean_absolute_error','mean_absolute_percentage_error'],
                          refit                 = True,
                          n_jobs                = 'auto',
                          suppress_warnings_fit = True,
                          verbose               = True,
                          show_progress         = True
                      )

print(f"Error backtest: {metric}")
predictions.head(4)

Information of backtesting process
----------------------------------
Number of observations used for initial training: 1977
Number of observations used for backtesting: 92
    Number of folds: 14
    Number of steps per fold: 7
    Number of steps to exclude from the end of each train set before test (gap): 0
    Last fold only includes 1 observations.

Fold: 0
    Training:   2017-12-01 00:00:00 -- 2023-04-30 00:00:00  (n=1977)
    Validation: 2023-05-01 00:00:00 -- 2023-05-07 00:00:00  (n=7)
Fold: 1
    Training:   2017-12-08 00:00:00 -- 2023-05-07 00:00:00  (n=1977)
    Validation: 2023-05-08 00:00:00 -- 2023-05-14 00:00:00  (n=7)
Fold: 2
    Training:   2017-12-15 00:00:00 -- 2023-05-14 00:00:00  (n=1977)
    Validation: 2023-05-15 00:00:00 -- 2023-05-21 00:00:00  (n=7)
Fold: 3
    Training:   2017-12-22 00:00:00 -- 2023-05-21 00:00:00  (n=1977)
    Validation: 2023-05-22 00:00:00 -- 2023-05-28 00:00:00  (n=7)
Fold: 4
    Training:   2017-12-29 00:00:00 -- 2023-05-28 00:00:00  (n=

  0%|          | 0/14 [00:00<?, ?it/s]

Error backtest: [21877800.9478089, 3511.8442544536765, 0.015925753712148452]


,pred
2023-05-01,196216.47
2023-05-02,219680.48
2023-05-03,221046.80
2023-05-04,221956.05
